# Interactive Viz

This is our implementation of an interactive vizualization showing differences between the grants amount accorded to university and institute, according to Canton, using P3 database of the SNSF for the grant data.

We use Folium to display the choropleth maps, Google places and GeoName API to link universities with cantons.

In [12]:
%matplotlib inline
import pandas as pd
import numpy as np
import scipy.stats as stats
import folium
import os
import json
import geopy
import geocoder
import time
import csv
from IPython.display import IFrame
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
#pip install python-google-places


First import the data

In [13]:
#dataCrow = pd.read_csv("data/players_cleaned_with_feature.csv", sep=',')

dataCrow = pd.read_csv("data/players_cleaned_with_feature_factorized.csv",sep=',')

In [62]:
dataCrow.ix[:18,:14]

,club,leagueCountry,birthday,position,redCards,meanIAT_redCards,ties,meanExp_yellowReds,yellowCards,meanIAT_yellowCards,meanExp_redCards,skin_color,height,meanExp_yellowCards
0,0,0,3598.0,0,0,-0.001420,179,-0.001615,19,-0.034618,-0.020220,0,182.0,-0.040843
1,1,1,6090.0,1,1,-0.000453,73,-0.001535,42,-0.002753,0.000029,0,183.0,-0.002366
2,2,0,6314.0,2,0,-0.001416,97,-0.001610,11,-0.035463,-0.020464,0,165.0,-0.042534
3,3,0,7664.0,3,1,-0.000147,42,-0.001747,31,-0.004654,-0.008712,0,178.0,-0.006586
4,4,2,7380.0,0,2,0.003984,40,0.010400,8,-0.022667,0.020312,0,180.0,-0.022177
5,5,2,6590.0,2,0,-0.001398,23,0.002186,11,-0.005505,-0.004792,1,180.0,-0.001384
6,6,2,8124.0,4,0,-0.001464,8,-0.001726,0,-0.045816,-0.002628,1,189.0,-0.057114
7,5,2,7535.0,4,0,-0.001485,1,-0.001573,0,-0.046475,-0.002618,1,188.0,-0.052035
8,7,2,7383.0,5,2,0.004065,25,-0.001622,21,0.014230,0.031430,1,180.0,0.019269
9,8,1,8948.0,6,1,0.005285,8,-0.001376,3,-0.025109,0.002929,1,179.0,-0.025370


In [15]:
dataCrow.ix[:8,15:]

,meanIAT_yellowReds,defeats,games,goals,weight
0,-0.001343,228,654,9,71.0
1,-0.001376,122,336,62,73.0
2,-0.001340,115,412,31,63.0
3,-0.001357,68,260,39,76.0
4,0.009606,43,124,1,73.0
5,0.002128,33,97,3,74.0
6,-0.001385,8,24,0,80.0
7,-0.001405,2,3,0,83.0
8,-0.001387,62,121,3,68.0


In [16]:
X=dataCrow
KMeans
KMeans(n_clusters=2, random_state=1).fit(X[X.columns[:12]])
print(np.mean(kmeans_model.labels_))

0.536997885835


In [17]:
#X = grouped
X=dataCrow
for i in range(2,15):
    kmeans_model = KMeans(n_clusters=2, random_state=1).fit(X[X.columns[:i]])
    labels = kmeans_model.labels_
    print(np.mean(labels))

0.523608174771
0.536997885835
0.536997885835
0.536997885835
0.536997885835
0.536997885835
0.536997885835
0.536997885835
0.536997885835
0.536997885835
0.536997885835
0.536997885835
0.536997885835


In [121]:
X=dataCrow.copy()
np.set_printoptions(threshold=np.nan)
dataCrow.size

28380

In [122]:
label_correct_color_skin = dataCrow.skin_color
label_correct_color_skin.shape

(1419,)

In [131]:
data_NoSkin = dataCrow.drop('skin_color',axis=1)
print(data_NoSkin.shape)
print(dataCrow.shape)

(1419, 19)
(1419, 20)


In [137]:
def clustering(X,label_correct_color_skin):
    kmeans_model = KMeans(n_clusters=2, random_state=0).fit(X)
    labels = kmeans_model.labels_
    silhouette_score_compute = silhouette_score(X, labels)
    labels_difference = label_correct_color_skin == labels
    
    #return labels, silhouette_score_compute, max(np.mean(labels_difference),1-np.mean(labels_difference))
    return silhouette_score_compute, max(np.mean(labels_difference),1-np.mean(labels_difference))
    

In [141]:
column = list(data_NoSkin.columns.values)
column

['club',
 'leagueCountry',
 'birthday',
 'position',
 'redCards',
 'meanIAT_redCards',
 'ties',
 'meanExp_yellowReds',
 'yellowCards',
 'meanIAT_yellowCards',
 'meanExp_redCards',
 'height',
 'meanExp_yellowCards',
 'victories',
 'meanIAT_yellowReds',
 'defeats',
 'games',
 'goals',
 'weight']

In [202]:
def FoundOptimum(data_NoSkin):
    columnOptSilhouette = 0
    columnOptClassSkin = 0
    valueOptSilhouette = 0
    valueOptClassSkin = 0

    for nameColumn in data_NoSkin.columns:
        currentData = data_NoSkin.drop(nameColumn,axis=1)
        currSilhouette, currClassSkin = clustering(currentData,dataCrow.skin_color)
        #print('drop columne : ',nameColumn,' currClassSkin : ',currClassSkin,'currSilhouette:',currSilhouette)

        if currClassSkin > valueOptClassSkin:
            valueOptClassSkin=currClassSkin
            columnOptClassSkin = nameColumn

        if currSilhouette > valueOptSilhouette:
            valueOptSilhouette = currSilhouette
            columnOptSilhouette = nameColumn

    return columnOptSilhouette, columnOptClassSkin, valueOptSilhouette, valueOptClassSkin
    #print(currentData.shape)

In [203]:
def optimizeSilhouette(data_No_Skin):
    
    
    currentData = data_No_Skin.copy()
    columnOptSilhouette, columnOptClassSkin = FoundOptimum(currentData)
    
    print('First value to delete found : ',columnOptSilhouette)
    
    currentData = currentData.drop(columnOptSilhouette,axis=1)
    columnOptSilhouette, columnOptClassSkin = FoundOptimum(currentData)
    
    print('Seconde value to delete found : ',columnOptSilhouette)
    
    currentData = currentData.drop(columnOptSilhouette,axis=1)
    columnOptSilhouette, columnOptClassSkin = FoundOptimum(currentData)
    
    print('Thrid value to delete found : ',columnOptSilhouette)
    currentData = currentData.drop(columnOptSilhouette,axis=1)
    
    
    return currentData, columnOptSilhouette, columnOptClassSkin 
    
    
    
    

In [210]:
def vraiOptimitez(data_No_Skin):
    
    currentData = data_No_Skin.copy()
    valueOptSilhouette=0
    valueOptClassSkin=0
    
    for i in range(1, 19):
        
        columnOptSilhouette, columnOptClassSkin,valueOptSilhouette,valueOptClassSkin = FoundOptimum(currentData)
        print('Iteration : ',i,' column to delete : ',columnOptSilhouette)
        currentData = currentData.drop(columnOptSilhouette,axis=1)
        
    return currentData, valueOptSilhouette, valueOptClassSkin

In [211]:
#cur, cols, colSk = optimizeSilhouette(data_NoSkin)
oData, optSilhouette, optClassSkin = vraiOptimitez(data_NoSkin)

Iteration :  1  column to delete :  games
Iteration :  2  column to delete :  victories
Iteration :  3  column to delete :  goals
Iteration :  4  column to delete :  defeats
Iteration :  5  column to delete :  club
Iteration :  6  column to delete :  ties
Iteration :  7  column to delete :  yellowCards
Iteration :  8  column to delete :  weight
Iteration :  9  column to delete :  height
Iteration :  10  column to delete :  position
Iteration :  11  column to delete :  redCards
Iteration :  12  column to delete :  birthday
Iteration :  13  column to delete :  meanExp_yellowCards
Iteration :  14  column to delete :  meanIAT_yellowCards
Iteration :  15  column to delete :  meanExp_redCards
Iteration :  16  column to delete :  meanExp_yellowReds
Iteration :  17  column to delete :  meanIAT_yellowReds
Iteration :  18  column to delete :  meanIAT_redCards


il faut optimiser 18 fois la silhouette et après une fois le classskin

In [212]:
#print(data_NoSkin.shape)
#columnOptSilhouette
#valueOptSilhouette
#print(forloop(data_NoSkin))
print('Silhouette ',optSilhouette,'class',optClassSkin)

Silhouette  0.751220995329 class 0.715292459479


In [213]:
oData.head()

,leagueCountry
0,0
1,1
2,0
3,0
4,2


In [101]:
X = dataCrow.drop('skin_color',axis=1)
label_correct_color_skin = dataCrow.skin_color
kmeanLabel, silScore, diff = clustering(X,label_correct_color_skin)


In [99]:
diff

0.51162790697674421